In [1]:
import warnings
warnings.filterwarnings('ignore')
from collections import defaultdict

import pandas as pd

Итоговый сервис будет тестироваться через REST API тестовыми кейсами. Требования к REST API для участников таковы:

1. REST API должен принимать данные в виде следующего JSON формата: {"question": "Как изменить пароль?"}.
2. В ответ мы ожидаем данные в JSON формате: {"answer": "Какой-то ответ", "class_1": "some_class", "class_2": "some_class"}.

In [2]:
import asyncio
from fastapi import FastAPI
from pydantic import BaseModel
import uvicorn

In [3]:
import json
import pickle
import os
from openai import OpenAI

from sentence_transformers import SentenceTransformer
import requests
import numpy as np
import faiss

def load_faiss_index():
    with open('./Model/chunks_data.pickle', 'rb') as handle: #Model/
        chunks = pickle.load(handle)

    model = SentenceTransformer('BAAI/bge-m3', device="cpu")
    embeddings = np.load('./Model/embs-data.npy')#Model/

    index = faiss.IndexFlatL2(embeddings.shape[1])  # build the index
    index.add(embeddings)

    return chunks, model, index, embeddings


def get_relevant_documents(question, chunks, model, index, embeddings):
    k = 3  # NUM of retrieval candidates
    e = model.encode(question)
    dist, Idx = index.search(e.reshape(1, -1), k)
    retrievals = [chunks[i] for i in Idx.flatten()]
    return retrievals

def respond_question(prompt):
    if prompt:
        # Check which model is selected and call the corresponding function
        chunks, model, index, embeddings = load_faiss_index()
        response, metadatas, rets = generate_mixtral_response(prompt, chunks, model, index,
                                                              embeddings)  # generate_mixtral_comment(question, answer, chunks, model, index, embeddings)#generate_mixtral_response(prompt, chunks, model, index, embeddings)
        # Process and display the response
        excerpts = '\n\n'.join([f"ИСТОЧНИК {i + 1}, {list(ret.metadata.keys())[0]}: {ret.page_content}" for i, ret in
                                enumerate(rets)]) if rets else ""
        bibliography = '\n\n'.join([f"{i + 1}. {meta}" for i, meta in enumerate(metadatas)]) if metadatas else ""
        full_response = f"{response}\n\n\n\nИСТОЧНИКИ:\n{excerpts}\n\nСПИСОК ЛИТЕРАТУРЫ:\n\n{bibliography}"
    return full_response  # f"{response}#\n\nИСТОЧНИКИ:\n{excerpts}\n\nСПИСОК ЛИТЕРАТУРЫ:\n\n{bibliography}"#full_response

def generate_mixtral_response(question, chunks, model, index, embeddings):
    rets = get_relevant_documents(question, chunks, model, index, embeddings)
    metadatas = [list(item.metadata.keys())[0] for item in rets]

    # Improved string formatting
    sources_text = ' \n\n '.join([f'ИСТОЧНИК {i + 1}: {ret.page_content}' for i, ret in enumerate(rets)])

    promptstring = (
        f"Вы оператор поддержки, который вежливо отвечает на вопросы пользователей на темы, связанные с видеохостингом. "
        f"Используя только информацию, содержащуюся в ИСТОЧНИКАХ после слова ТЕКСТ, "
        f"ответьте на вопрос, заданный после слова ВОПРОС. "
        f"Если в тексте нет информации, необходимой для ответа, ответьте «Недостаточно информации для ответа». "
        f"Структурируйте свой ответ и отвечайте шаг за шагом, но не упоминайте ссылки на источники в ответе."
        f"Проверьте, что ответ выводится на кириллице и на русском языке. \n"
        f"ТЕКСТ:\n{sources_text}\nВОПРОС:\n{question}"
    )

   
# Set OpenAI's API key and API base to use vLLM's API server.
    openai_api_key = "EMPTY"
    openai_api_base = "http://localhost:8000/v1"
    
    client = OpenAI(
        api_key=openai_api_key,
        base_url=openai_api_base,
    )
    
    chat_response = client.chat.completions.create(
        model="mistralai/Mistral-7B-Instruct-v0.3",
        messages=[
            
            {"role": "user", "content": promptstring},
            
        ],
        max_tokens = 2048,
    )
    print("Chat response:", chat_response)

    return chat_response.choices[0].message.content, metadatas, rets


In [4]:
class Request(BaseModel):
    question: str


class Response(BaseModel):
    answer: str
    class_1: str
    class_2: str

app = FastAPI()


@app.get("/")
def index():
    return {"text": "Интеллектуальный помощник оператора службы поддержки."}

    
@app.post("/predict")
async def predict_sentiment(request: Request):
    question = request.question
  
    answer_llm = respond_question(question)
  
    response = Response(
        answer=answer_llm,#anwer_data['Ответ из БЗ'],
        class_1='',#anwer_data['Классификатор 1 уровня'], # Классификатор оценивается опционально; при отсутствии можно задать константное значение.
        class_2=''#anwer_data['Классификатор 2 уровня'], # Классификатор оценивается опционально; при отсутствии можно задать константное значение.
    )
    
  
    return response




if __name__ == "__main__":
    host = "213.171.28.90" # Сконфигурируйте host согласно настройкам вашего сервера.
    config = uvicorn.Config(app, host=host, port=8001)#8000
    server = uvicorn.Server(config)
    loop = asyncio.get_running_loop()
    loop.create_task(server.serve())


INFO:     Started server process [13358]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://213.171.28.90:8001 (Press CTRL+C to quit)


Chat response: ChatCompletion(id='chat-4b2bc72b8d7b45b1820c8d66fa7f11b8', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content=' Чтобы сменить пароль на видеохостинге RUTUBE, вам необходимо выполнить следующие шаги:\n\n1. Авторизуйтесь на сайте RUTUBE (https://rutube.ru).\n2. В меню профиля найдите пункт "Изменить пароль" и нажмите на него.\n3. Следуйте инструкциям и заполните поле для нового пароля.\n4. Подтвердите изменение пароля в поле для подтверждения нового пароля.\n5. Нажмите кнопку "Сохранить" или "Обновить" для сохранения изменений.\n\nВажно обратить внимание, что эти шаги могут отличаться в зависимости от устройства (например, в мобильном приложении будет другой способ изменения пароля). Если у вас возникнут трудности с изменением пароля, я рекомендую пройтись по шагам снова, либо обратиться в службу поддержки RUTUBE.', refusal=None, role='assistant', function_call=None, tool_calls=[]), stop_reason=None)], created=1727592403, mo

# Для проверки того, что REST API отвечает корректно, запустите данный код в отдельном ноутбуке или Python-скрипте.